<a href="https://colab.research.google.com/github/georgialoukatou/childes_bert_tagging/blob/main/bert_pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://pageperso.lis-lab.fr/benoit.favre/pstaln/09_embedding_evaluation.html
#https://colab.research.google.com/drive/1sbfjIapc1MDcQpGYlLkapjhWWv3yOR9t#scrollTo=hW7aY1VWtbzE
#https://colab.research.google.com/drive/1PHv-IRLPCtv7oTcIGbsgZHqrB5LPvB7S#scrollTo=IdU4YVqb7N8M

!pip install -qq transformers
!pip -q install conllu

import conllu
import pandas as pd
import numpy as np
import urllib.request
import time
import torch
import re
import collections
import torch.optim as optim
import torch.nn as nn
import sys


from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel


     |████████████████████████████████| 3.4 MB 14.2 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
     |████████████████████████████████| 3.3 MB 46.5 MB/s 


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
#torch.cuda.is_available()
#torch.cuda.get_device_name(0)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

!pip install ipython-autotime
%load_ext autotime

time: 127 µs (started: 2022-01-23 01:30:42 +00:00)


In [ ]:
#import english hand annotated CHILDES sample !!!! space added before 'd, n't, 's, 'm, _, 'v (same as uds)
eng_gold_childes = pd.read_csv('https://raw.githubusercontent.com/georgialoukatou/childes_bert_tagging/main/eng_gold_childes_clean_short_.csv')
eng_gold_childes = eng_gold_childes.dropna(axis=0, subset=['correct_pos']) 
mapping = {'mod': 'AUX', 'prep': 'ADP','adj': 'ADJ', 'pro': 'PRON', 'v': 'VERB', 'adv': 'ADV', 'participle': 'VERB', 'n': 'NOUN', 'childes': 'X', 'nan': 'X', 'intj': 'INTJ', 'particle': 'PART', 'mod': 'AUX','mod ': 'AUX', 'spacy': 'X', 'on': 'X', 'participle': 'VERB', 'aux': 'AUX', 'num': 'NUM', 'det':'DET' }
#for English, missing PROPN, CCONJ, SCONJ, SYM, PUNCT
eng_gold_childes = eng_gold_childes.replace({'correct_pos': mapping})
eng_gold_childes_test = eng_gold_childes[['utterance_gloss','correct_pos', 'position']]
eng_gold_childes_test['utterance_gloss']=eng_gold_childes_test['utterance_gloss'].str.lower()
print(eng_gold_childes_test.head())
print(eng_gold_childes_test.correct_pos.unique())
print(len(eng_gold_childes_test))

                                     utterance_gloss correct_pos  position
0  the shortcut led right to where harold thought...         AUX      11.0
1  i'm not allowing you to reach things onto the ...         ADP       9.0
2                                      i dare not go         AUX       1.0
3                        yyy knee behind the letters         ADP       4.0
4                    is that kind of a greenish blue         ADJ       5.0
['AUX' 'ADP' 'ADJ' 'PRON' 'VERB' 'ADV' 'PROPN' 'NOUN' 'X' 'NUM' 'INTJ'
 'PART']
882
time: 203 ms (started: 2022-01-23 04:44:16 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for filename in ['en_ewt-ud-train.conllu', 'en_ewt-ud-dev.conllu', 'en_ewt-ud-test.conllu']:
  urllib.request.urlretrieve('https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/' + filename, filename)

def load_conllu(filename):
  with open(filename) as fp:
    data = conllu.parse(fp.read())
  sentences = [[token['form'] for token in sentence] for sentence in data]
  taggings = [[token['upos'] for token in sentence] for sentence in data]
  return sentences, taggings

train_sentences, train_taggings = load_conllu('en_ewt-ud-train.conllu')
valid_sentences, valid_taggings = load_conllu('en_ewt-ud-dev.conllu')
test_sentences, test_taggings = load_conllu('en_ewt-ud-test.conllu')


time: 8.04 s (started: 2022-01-23 01:30:43 +00:00)


In [ ]:
ud_train=pd.DataFrame(columns=list('AB'))
for i in zip(train_sentences, train_taggings):
  ud_train = ud_train.append({'A': i[0],'B': i[1]}, ignore_index=True)

ud_valid=pd.DataFrame(columns=list('AB'))
for i in zip(valid_sentences, valid_taggings):
  ud_valid = ud_valid.append({'A': i[0],'B': i[1]}, ignore_index=True)

ud_test=pd.DataFrame(columns=list('AB'))
for i in zip(test_sentences, test_taggings):
  ud_test = ud_test.append({'A': i[0],'B': i[1]}, ignore_index=True)

ud_train.tail()


,A,B
12538,"[Of, course, ,, they, couldn't, could, n't, ca...","[ADP, NOUN, PUNCT, PRON, _, AUX, PART, VERB, P..."
12539,"[On, Monday, I, called, and, again, it, was, a...","[ADP, PROPN, PRON, VERB, CCONJ, ADV, PRON, AUX..."
12540,"[Supposedly, they, will, be, holding, it, for,...","[ADV, PRON, AUX, AUX, VERB, PRON, ADP, PRON, D..."
12541,"[The, employees, at, this, Sear's, are, comple...","[DET, NOUN, ADP, DET, PROPN, AUX, ADV, ADJ, CC..."
12542,"[I, will, never, return, there, again, (, and,...","[PRON, AUX, ADV, VERB, ADV, ADV, PUNCT, CCONJ,..."


time: 33.3 s (started: 2022-01-23 01:30:51 +00:00)


In [ ]:
ud_test_clean=pd.DataFrame(columns=list('AB'))
ud_train_clean=pd.DataFrame(columns=list('AB'))
ud_valid_clean=pd.DataFrame(columns=list('AB'))

def remove_(df, dfnew):
  for index, row in df.iterrows():
    new_r=[]
    new_p=[]
    for i, r in enumerate(row['B']):
      if r !="_":
        new_p.append(r)
        new_r.append(row['A'][i])
    dfnew = dfnew.append({'A': new_r,'B': new_p}, ignore_index=True)
  return dfnew

ud_train_new=remove_(ud_train, ud_train_clean)
ud_train=ud_train_new

ud_test_new=remove_(ud_test, ud_test_clean)
ud_test=ud_test_new

ud_valid_new=remove_(ud_valid, ud_valid_clean)
ud_valid=ud_valid_new

print(ud_train.tail())

                                                       A                                                  B
12538  [Of, course, ,, they, could, n't, call, him, e...  [ADP, NOUN, PUNCT, PRON, AUX, PART, VERB, PRON...
12539  [On, Monday, I, called, and, again, it, was, a...  [ADP, PROPN, PRON, VERB, CCONJ, ADV, PRON, AUX...
12540  [Supposedly, they, will, be, holding, it, for,...  [ADV, PRON, AUX, AUX, VERB, PRON, ADP, PRON, D...
12541  [The, employees, at, this, Sear's, are, comple...  [DET, NOUN, ADP, DET, PROPN, AUX, ADV, ADJ, CC...
12542  [I, will, never, return, there, again, (, and,...  [PRON, AUX, ADV, VERB, ADV, ADV, PUNCT, CCONJ,...
time: 35.9 s (started: 2022-01-23 01:31:24 +00:00)


In [ ]:
# load tokenizer for a specific bert model (bert-base-cased)

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # tokenizer from English model better for english than multilingual
# tokenize an example sentence
tokenizer.tokenize('This tokenizer is sooooo awesome.')

# load a specific bert model (bert-base-cased)
bert=BertModel.from_pretrained('bert-base-uncased')

# tensor length
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
max_input_length

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


512

time: 51.8 s (started: 2022-01-23 01:32:00 +00:00)


In [ ]:
uniq_taggings=[]
for i in ud_train['B']:
  for y in i:
    if y not in uniq_taggings:
      uniq_taggings.append(y)
print(uniq_taggings)

['PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADP', 'AUX', 'PRON', 'PART', 'SCONJ', 'NUM', 'ADV', 'CCONJ', 'X', 'INTJ', 'SYM']
time: 63.5 ms (started: 2022-01-23 01:32:52 +00:00)


In [ ]:
#ud_train['A'][41]
ud_train.loc[[10010]]


,A,B
10010,"[I, also, understand, that, weekend, staffs, a...","[PRON, ADV, VERB, SCONJ, NOUN, NOUN, AUX, ADJ,..."


time: 14.9 ms (started: 2022-01-23 01:32:52 +00:00)


In [ ]:
#for sentence,tagging in zip(df_long['A'],df_long['B']):
 # tokenized_s = tokenizer.tokenize(' '.join(sentence))

time: 758 µs (started: 2022-01-23 01:32:52 +00:00)


In [ ]:
def align_tokenizations(sentences, taggings):
  bert_tokenized_sentences = []
  aligned_taggings = []

  for sentence, tagging in zip(sentences, taggings):
    sentence = (map(lambda x: x.lower(), sentence))
    sentence = list(sentence)
    # first generate BERT-tokenization
    bert_tokenized_sentence = tokenizer.tokenize(' '.join(sentence))
    aligned_tagging = []
    current_word = ''
    index = 0 # index of current word in sentence and tagging
    for token in bert_tokenized_sentence:
      current_word += re.sub(r'^##', '', token) # recompose word with subtoken
      sentence[index] = sentence[index].replace('\xad', '') # fix bug in data
      # note that some word factors correspond to unknown words in BERT
      assert token == '[UNK]' or sentence[index].startswith(current_word)
      if token == '[UNK]' or sentence[index] == current_word: # if we completed a word
        current_word = ''
        aligned_tagging.append(tagging[index])
        index += 1
      else: # otherwise insert padding
        aligned_tagging.append('<pad>')

    assert len(bert_tokenized_sentence) == len(aligned_tagging)

    bert_tokenized_sentences.append(bert_tokenized_sentence)
    aligned_taggings.append(aligned_tagging)

  return bert_tokenized_sentences, aligned_taggings

time: 22.6 ms (started: 2022-01-23 01:32:52 +00:00)


In [ ]:
#remove sentences with problematic formatting
ud_train= ud_train.drop([157, 530, 7132])
ud_train.drop(ud_train.index[8207:8210], inplace=True)
ud_train.drop(ud_train.index[9974], inplace=True)

#align tokenized sentences and tagging for train
bert_tokenized_sentences, aligned_taggings = align_tokenizations(ud_train['A'], ud_train['B'])
len(ud_train)

12536

time: 5.6 s (started: 2022-01-23 01:32:52 +00:00)


In [ ]:
bert_tokenized_sentences_test, aligned_taggings_test = align_tokenizations(ud_test['A'], ud_test['B'])

ud_valid = ud_valid.drop([97, 1622])
bert_tokenized_sentences_valid, aligned_taggings_valid = align_tokenizations(ud_valid['A'], ud_valid['B'])


time: 1.43 s (started: 2022-01-23 01:32:58 +00:00)


In [ ]:
for i in zip(bert_tokenized_sentences, aligned_taggings):
  print(i)

In [ ]:
label_vocab = collections.defaultdict(lambda: len(label_vocab))
label_vocab['<pad>'] = 0

def convert_to_ids(sentences, taggings):
  sentences_ids = []
  taggings_ids = []
  for sentence, tagging in zip(sentences, taggings):
    sentence_tensor = torch.tensor(tokenizer.convert_tokens_to_ids(['[CLS]'] + sentence + ['[SEP]'])).long()
    tagging_tensor = torch.tensor([0] + [label_vocab[tag] for tag in tagging] + [0]).long()
    sentences_ids.append(sentence_tensor)   #sentences_ids.append(sentence_tensor.to(device))
    taggings_ids.append(tagging_tensor)     #taggings_ids.append(tagging_tensor.to(device))

  return sentences_ids, taggings_ids

#convert train sentences to ids and tensors
train_sentences_ids, train_taggings_ids = convert_to_ids(bert_tokenized_sentences, aligned_taggings)
#same for test, dev
test_sentences_ids, test_taggings_ids = convert_to_ids(bert_tokenized_sentences_test, aligned_taggings_test)
valid_sentences_ids, valid_taggings_ids = convert_to_ids(bert_tokenized_sentences_valid, aligned_taggings_valid)

output_dim=len(label_vocab) #dimensions of labels

time: 1.54 s (started: 2022-01-23 01:33:00 +00:00)


In [ ]:
for tag, count in sorted(label_vocab.items(), reverse=True, key=lambda x: x[1]):
  print(count, tag)

17 SYM
16 INTJ
15 X
14 CCONJ
13 ADV
12 NUM
11 SCONJ
10 PART
9 PRON
8 AUX
7 ADP
6 DET
5 VERB
4 NOUN
3 ADJ
2 PUNCT
1 PROPN
0 <pad>
time: 4.6 ms (started: 2022-01-23 01:33:01 +00:00)


In [ ]:
#function to be fed to the collate_fn parameter of dataloader showing how to glue data

def collate_fn(items):
  #max_len = max(len(item[0]) for item in items)
  max_len=max_input_length
  sentences = torch.zeros((len(items), max_len), device=items[0][0].device).long().to(device)
  taggings = torch.zeros((len(items), max_len)).long().to(device)

  for i, (sentence, tagging) in enumerate(items):
    sentences[i][0:len(sentence)] = sentence
    taggings[i][0:len(tagging)] = tagging

  return sentences, taggings

#example
x, y = collate_fn([[torch.tensor([1, 2, 3]), torch.tensor([4, 5, 6])], [torch.tensor([1, 2]), torch.tensor([3, 4])]])
print(x.shape, y.shape)

torch.Size([2, 512]) torch.Size([2, 512])
time: 10.2 s (started: 2022-01-23 01:33:01 +00:00)


In [ ]:
class PosTaggingDataset(Dataset):
  def __init__(self, sentences, taggings):
    assert len(sentences) == len(taggings)
    self.sentences = sentences
    self.taggings = taggings

  def __getitem__(self, i):
    return self.sentences[i], self.taggings[i]

  def __len__(self):
    return len(self.sentences)

time: 3 ms (started: 2022-01-23 01:33:11 +00:00)


In [ ]:
example=PosTaggingDataset(train_sentences_ids, train_taggings_ids)[0:2]
example

([tensor([  101,  2632,  1011, 23564,  2386,  1024,  2137,  2749,  2730, 21146,
          28209, 14093,  2632,  1011,  2019,  2072,  1010,  1996, 14512,  2012,
           1996,  8806,  1999,  1996,  2237,  1997,  1053,  4886,  2213,  1010,
           2379,  1996,  9042,  3675,  1012,   102]),
  tensor([  101,  1031,  2023,  4288,  1997,  1037,  9768, 29307,  2097,  2022,
           4786,  2149,  4390,  2005,  2086,  2000,  2272,  1012,  1033,   102])],
 [tensor([0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 1, 1, 2, 0, 1, 2, 6, 4, 7, 6, 4, 7, 6,
          4, 7, 0, 0, 1, 2, 7, 6, 3, 4, 2, 0]),
  tensor([ 0,  2,  6,  4,  7,  6,  3,  4,  8,  8,  5,  9,  4,  7,  4, 10,  5,  2,
           2,  0])])

time: 10.2 ms (started: 2022-01-23 01:33:11 +00:00)


In [ ]:
#from sklearn.model_selection import train_test_split
#RANDOM_SEED=1660

df_train = PosTaggingDataset(train_sentences_ids, train_taggings_ids)

df_test = PosTaggingDataset(test_sentences_ids, test_taggings_ids)

df_valid = PosTaggingDataset(valid_sentences_ids, valid_taggings_ids)

print(len(df_train), len(df_test), len(df_valid))

12536 2077 1999
time: 3.61 ms (started: 2022-01-23 01:33:11 +00:00)


In [ ]:
batch_size=8  ######### TO MODIDY

train_loader = DataLoader(df_train, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)
test_loader = DataLoader(df_test, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)
valid_loader = DataLoader(df_valid, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

time: 3.2 ms (started: 2022-01-23 01:33:11 +00:00)


In [ ]:
next(iter(train_loader))[0][7]

tensor([  101,  1996,  4442,  2020,  4082,  1999,  1996,  1043,  3270, 16739,
         8717,  2232,  1998,  2632,  1011, 24815,  4733,  1997,  1996,  3007,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

time: 26 ms (started: 2022-01-23 01:33:11 +00:00)


In [ ]:
##probing: https://nlp.stanford.edu/~johnhew/interpreting-probes.html
#A probe is trained to predict properties we care about from representations of a model whose nature we'd like to know more about.

class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):     
        super().__init__()       
        self.bert = bert      
        embedding_dim = bert.config.to_dict()['hidden_size']       
        self.fc = nn.Linear(embedding_dim, output_dim)      
        self.dropout = nn.Dropout(dropout)       
    def forward(self, text): 
        #text = [sent len, batch size] 
        text = text.permute(1, 0)     
        #text = [batch size, sent len]    
        embedded = self.dropout(self.bert(text)[0])     
        #embedded = [batch size, seq len, emb dim]             
        embedded = embedded.permute(1, 0, 2)                
        #embedded = [sent len, batch size, emb dim]   
        predictions = self.fc(self.dropout(embedded))    
        #predictions = [sent len, batch size, output dim]  
        return predictions

time: 9 ms (started: 2022-01-23 01:33:11 +00:00)


In [ ]:
# check that model works on an arbitrary batch that contains two sentences of length 3
dropout=0
bert_linear_predict = BERTPoSTagger(bert,output_dim,dropout)

#example
with torch.no_grad():
  y = bert_linear_predict(torch.tensor([[0, 1, 2], [3, 4, 5]]))
#the expected shape is (batch size, max sentence length, number of labels)
print(y.shape)
print(y)


torch.Size([2, 3, 18])
tensor([[[-0.1971,  0.1687,  0.1270, -0.2264, -0.2632,  0.4973,  0.0043,
           0.3420,  0.0573, -0.4201, -0.0877,  0.0175, -0.5222, -0.6205,
          -0.1678, -0.3459,  0.1605,  0.6933],
         [ 0.1106,  0.2798, -0.0662,  0.1927, -0.0305, -0.1726,  0.0262,
          -0.4546,  0.0319, -0.2883, -0.2684,  0.3507,  0.1152, -0.9065,
           0.1733, -0.0897, -0.1337,  0.0567],
         [-0.1782,  0.2660,  0.0813, -0.3510, -0.1974,  0.5941,  0.0816,
           0.2911,  0.0312, -0.2641, -0.0701,  0.0752, -0.6061, -0.4124,
          -0.0933, -0.2160,  0.2038,  0.5588]],

        [[-0.2059, -0.0597,  0.3319, -0.0901, -0.2668,  0.7209, -0.0350,
           0.3921, -0.0768, -0.3513, -0.2652, -0.1082, -0.4685, -0.4472,
          -0.2191, -0.1744,  0.1445,  0.6940],
         [-0.2075,  0.1183, -0.1732,  0.2249,  0.1816,  0.4791, -0.0516,
           0.0762, -0.1033, -0.3017, -0.0672,  0.1433, -0.2133,  0.1493,
          -0.0714, -0.1559, -0.1094,  0.2901],
         [

In [ ]:
#load Eva's model
#childes_bert = torch.load('model.pt')
#childes_bert.eval()

time: 567 µs (started: 2022-01-23 01:33:12 +00:00)


In [ ]:
bert_linear_predict = BERTPoSTagger(bert,output_dim,dropout)

bert_linear_predict = bert_linear_predict.to(device)

time: 164 ms (started: 2022-01-23 01:33:12 +00:00)


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(bert_linear_predict):,} trainable parameters')

The model has 109,496,082 trainable parameters
time: 2.67 ms (started: 2022-01-23 01:33:12 +00:00)


In [ ]:
TAG_PAD_IDX = 0

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX) #for multi class classification task

def perf(model, loader):
  model.eval() # do not apply training-specific steps such as dropout
  total_loss = correct = num_loss = num_perf = 0
  for x, y in loader:
    with torch.no_grad(): # no need to store computation graph for gradients
      # perform inference and compute loss
      y_scores = model(x)
      loss = criterion(y_scores.view(-1, output_dim), y.view(-1)) # requires tensors of shape (num-instances, num-labels) and (num-instances)

      # gather loss statistics
      total_loss += loss.item() #the loss is usually negative log-likelihood and residual sum of squares for classification and regression respectively. Then naturally, the main objective in a learning model is to reduce (minimize) the loss function's value with respect to the model's parameters by changing the weight vector values through different optimization methods, such as backpropagation in neural networks. Loss value implies how well or poorly a certain model behaves after each iteration of optimization. Ideally, one would expect the reduction of loss after each, or several, iteration(s).

      num_loss += 1

      # gather accuracy statistics
      y_pred = torch.max(y_scores, 2)[1] # compute highest-scoring tag
      mask = (y != 0) # ignore <pad> tags
      correct += torch.sum((y_pred == y) * mask) # compute number of correct predictions
      num_perf += torch.sum(mask).item()
  return total_loss / num_loss, correct.item() / num_perf

# without training, accuracy should be a bit less than 2% (chance of getting a label correct)


time: 11.4 ms (started: 2022-01-23 01:33:12 +00:00)


In [ ]:
print('BERT representation (unsupervised) on train dataset (without training)', *perf(bert_linear_predict, train_loader))
print('BERT representation (unsupervised) on validation dataset (without training)', *perf(bert_linear_predict, valid_loader))
print('BERT representation (unsupervised) on test dataset (without training)', *perf(bert_linear_predict, test_loader))

BERT representation (unsupervised) on train dataset (without training) 2.9386303055111433 0.037933817594834544
BERT representation (unsupervised) on validation dataset (without training) 2.945868305206299 0.03979465955668749
BERT representation (unsupervised) on test dataset (without training) 2.9409543871879578 0.04051954261125941
time: 8min 5s (started: 2022-01-23 01:33:12 +00:00)


In [ ]:
#need to Fine-tune a BERT model for POS-tagging in the fully supervised setting (use a learning rate of 2e-5, and use a linear warmup learning rate schedule on 10% of the updates as suggested by BERT authors)

#Next, we define our optimizer. Usually when fine-tuning you want to use a lower learning rate than normal, this is because we don't want to drastically change the parameters as it may cause our model to forget what it has learned. This phenomenon is called catastrophic forgetting.

#We pick 5e-5 (0.00005) as it is one of the three values recommended in the BERT paper. Again, there may be better values for this dataset.

time: 1.54 ms (started: 2022-01-23 01:41:18 +00:00)


In [ ]:
#Training is very similar to evaluation as it also performs inference. In addition it uses an optimizer which modifies the parameters of the neural network to minimize the criterion thanks to the gradients accumulated through the forward pass of the model. At each epoch, we perform inference, modify model weights after each batch, and finally use perf to compute loss and accuracy on the validatin data.
#Note that training is successful when the training loss gets lower after every epoch. It might fluctuate on validation data because of overtraining or generalization noise.

LEARNING_RATE = 5e-5 #### TO MODIFY

optimizer = optim.Adam(bert_linear_predict.parameters(), lr = LEARNING_RATE)

def fit(model, epochs):
  best_valid_loss = float('inf')
  for epoch in range(epochs):
    model.train()
    total_loss = num = correct = num_loss = num_perf = 0
    start_time = time.time()
    for x, y in train_loader:

      optimizer.zero_grad() # start accumulating gradients
      y_scores = model(x)
      loss = criterion(y_scores.view(-1, output_dim), y.view(-1))
      loss.backward() # compute gradients though computation graph
      optimizer.step() # modify model parameters
      total_loss += loss.item()
      num += 1
      y_pred = torch.max(y_scores, 2)[1] # compute highest-scoring tag
      mask = (y != 0) # ignore <pad> tags
      correct += torch.sum((y_pred == y) * mask) # compute number of correct predictions
      num_perf += torch.sum(mask).item()

    train_loss=total_loss / num 
    train_acc= correct.item() / num_perf
      
    end_time = time.time()  
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    valid_loss, valid_acc = perf(model, valid_loader)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut-model.pt')

    #print(1 + epoch, total_loss / num, *perf(model, valid_loader))# valid loader instead of train loader SOS SOS
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(1 + epoch, f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(1 + epoch, f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

time: 24.7 ms (started: 2022-01-23 01:41:18 +00:00)


In [ ]:
torch.cuda.empty_cache() 
#torch.cuda.memory_summary(device=None, abbreviated=False)

fit(bert_linear_predict, 2)

Epoch: 01 | Epoch Time: 17m 33s
1 	Train Loss: 0.520 | Train Acc: 82.87%
1 	 Val. Loss: 0.439 |  Val. Acc: 86.48%
Epoch: 02 | Epoch Time: 17m 33s
2 	Train Loss: 0.326 | Train Acc: 89.19%
2 	 Val. Loss: 0.408 |  Val. Acc: 87.23%
time: 37min 11s (started: 2022-01-23 01:41:18 +00:00)


In [ ]:
#for batch in iter(train_loader):
  #print(batch)

time: 1.02 ms (started: 2022-01-23 02:18:29 +00:00)


In [ ]:
saved_model = BERTPoSTagger(bert,output_dim,dropout)

saved_model = saved_model.to(device)

saved_model.load_state_dict(torch.load('tut-model.pt'))

test_loss, test_acc = perf(saved_model, test_loader)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.391 | Test Acc: 87.78%
time: 1min 3s (started: 2022-01-23 02:18:29 +00:00)


In [ ]:
td=[]
to_tokens=[]

s="smell the earth and the trees after warm summer showers breath in the sweet smell from a field full of flowers"
for i in list(s.split(" ")):
  td.append(i)
#to_ids= tokenizer.convert_tokens_to_ids(["[CLS]","and", "house", "go","[SEP]"])
tok = tokenizer.tokenize(' '.join(td))
print(tok)

['smell', 'the', 'earth', 'and', 'the', 'trees', 'after', 'warm', 'summer', 'showers', 'breath', 'in', 'the', 'sweet', 'smell', 'from', 'a', 'field', 'full', 'of', 'flowers']
['smell', 'the', 'earth', 'and', 'the', 'trees', 'after', 'warm', 'summer', 'showers', 'breath', 'in', 'the', 'sweet', 'smell', 'from', 'a', 'field', 'full', 'of', 'flowers']
time: 5.38 ms (started: 2022-01-23 02:23:24 +00:00)


In [ ]:
ex=example[0]


to_ids= tokenizer.convert_tokens_to_ids(["[CLS]","and", "house", "go","[SEP]"])
print(to_ids)

ex=torch.tensor([to_ids]).to(device)

[101, 1998, 2160, 2175, 102]
time: 3.82 ms (started: 2022-01-23 02:23:51 +00:00)


In [ ]:
def collate_gold(items):
  sentences = torch.zeros((len(items), max_len), device=items[0][0].device).long().to(device)
  #taggings = torch.zeros((len(items), max_len)).long().to(device)

  for i, sentence in enumerate(items):
    sentences[i][0:len(sentence)] = sentence
    #taggings[i][0:len(tagging)] = tagging

  return sentences#, taggings



time: 2.85 ms (started: 2022-01-23 02:23:53 +00:00)


In [ ]:
label_vocab_inv = {y:x for x,y in label_vocab.items()}

def tag_sentence(model, s): #tokenizer,tag_field,text_field, device,   
    model.eval()
    with torch.no_grad():
      y = saved_model(s)
      top_predictions = y.argmax(-1)
    return top_predictions
  
output_=tag_sentence(saved_model, ex)
res_c=output_.cpu().numpy()[0][1:-1]
print(list(label_vocab_inv[f] for f in res_c)) 


['CCONJ', 'NOUN', 'VERB']
time: 29.2 ms (started: 2022-01-23 02:23:55 +00:00)


In [ ]:
#test on hand annotations
orig_stdout = sys.stdout
f = open('out.txt', 'w') #save results here
sys.stdout = f

df=[]
for sentence in eng_gold_childes_test['utterance_gloss']:
  word=sentence.split()
  df.append(word)

for s in df:
  tokenized_s = tokenizer.tokenize(' '.join(s))
  sen_ = tokenizer.convert_tokens_to_ids(['[CLS]'] + tokenized_s + ['[SEP]'])
  gold=torch.tensor([sen_]).to(device)
  res=tag_sentence(saved_model, gold)
  res_c=res.cpu().numpy()[0][1:-1]
  print(f'{tokenized_s} -- {list(label_vocab_inv[f] for f in res_c)}')

output=[]
sys.stdout = orig_stdout
f.close()   

time: 9.88 s (started: 2022-01-23 04:44:28 +00:00)


In [ ]:
#evaluate human annotations

#df = df.dropna(axis=0, subset=['correct_pos']) 
mapping = {'mod': 'AUX', 'prep': 'ADP','adj': 'ADJ', 'pro': 'PRON', 'v': 'VERB', 'adv': 'ADV', 'participle': 'VERB', 'n': 'NOUN', 'childes': 'X', 'nan': 'X', 'intj': 'INTJ', 'particle': 'PART', 'mod': 'AUX','mod ': 'AUX', 'spacy': 'X', 'on': 'X', 'participle': 'VERB', 'aux': 'AUX', 'num': 'NUM', 'det':'DET' }
#for English, missing PROPN, CCONJ, SCONJ, SYM, PUNCT
correct_labels = eng_gold_childes_test.replace({'correct_pos': mapping})
correct_labels = correct_labels[['correct_pos', 'position']]
correct_labels

,correct_pos,position
0,AUX,11.0
1,ADP,9.0
2,AUX,1.0
3,ADP,4.0
4,ADJ,5.0
...,...,...
901,PART,7.0
902,PROPN,2.0
903,PART,4.0
904,PRON,1.0


time: 28.5 ms (started: 2022-01-23 04:44:38 +00:00)


In [ ]:
f = open('out.txt', 'r')

correct=0
c=0
for ind, line in enumerate(f):
    c+=1
    #print(ind,line)
    words=line.split("--")[0]
    tags=line.split("--")[1]
    w=list(x.replace("[","").replace(" ", "").replace("\n","").replace("]","") for x in words.split(","))
    l=list(x.replace("[","").replace(" ", "").replace("\n","").replace("]","") for x in tags.split(","))
    position=int(correct_labels.iloc[ind]['position'])
    #print(position, c)
    #print(position, ind)
    print(c,w[position][1:-1], l[position][1:-1],correct_labels.iloc[ind]['correct_pos'])
    if l[position][1:-1]== correct_labels.iloc[ind]['correct_pos']:
      correct+=1
acc=correct/len(df)*100  
print(f'Accuracy on English gold CHILDES: {correct} / {len(df)} , {acc:.2f}%')

1 ought VERB AUX
2 onto ADP ADP
3 dare NOUN AUX
4 behind ADP ADP
5 greenish ADJ ADJ
6 i PRON PRON
7 wrote VERB VERB
8 must AUX AUX
9 anybody ADV PRON
10 grand ADJ ADJ
11 suddenly ADV ADV
12 want VERB VERB
13 hiding VERB VERB
14 must AUX AUX
15 ali PROPN PROPN
16 good ADJ ADJ
17 fellows NOUN NOUN
18 of ADP ADP
19 grass NOUN NOUN
20 tip NOUN ADJ
21 yu NOUN ADJ
22 wanted VERB VERB
23 down INTJ ADV
24 from ADP ADP
25 per ADP ADJ
26 visited VERB VERB
27 good ADJ ADJ
28 good ADJ ADJ
29 off ADP ADP
30 wild ADJ ADJ
31 block NOUN NOUN
32 there PRON PRON
33 where ADV PRON
34 narrow ADJ ADJ
35 onto ADP ADP
36 tap NOUN VERB
37 might AUX AUX
38 themselves PRON PRON
39 have AUX VERB
40 which PRON PRON
41 making VERB VERB
42 you PRON PRON
43 got VERB VERB
44 down INTJ ADV
45 here ADV ADV
46 born VERB ADJ
47 mighty ADV ADV
48 closing VERB VERB
49 happily ADV ADV
50 take VERB VERB
51 shall AUX AUX
52 feeling NOUN NOUN
53 everybody PRON PRON
54 from ADP ADP
55 hydra NOUN NOUN
56 little ADV ADJ
57 respon